데이터 셋 no_arrow_data로 고정 후 파라미터 변경

# pip, install

In [1]:
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/huggingface/peft.git

!pip install openai
!pip install scipy
!pip install spacy-transformers

!pip install bitsandbytes
!python -m bitsandbytes
!pip install protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 54.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 3.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 56.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 102.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 111.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.3/913.3 kB 117.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 153.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 63.8 MB/s eta 0:00:00
    

In [2]:
import os

import torch
import bitsandbytes
from datasets import load_dataset
import transformers
from transformers import LlamaTokenizer, AutoTokenizer, AutoConfig, GenerationConfig, AutoModelForCausalLM

from peft import prepare_model_for_kbit_training


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/lib/x86_64-linux-gnu did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('noninteractive    SHELL=/bin/bash')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/workspace/Untitled.ipy

In [3]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:           1.0Ti       166Gi       237Gi       306Mi       603Gi       834Gi
Swap:             0B          0B          0B


# 모델과 토크나이저

In [4]:
model_path = 'openlm-research/open_llama_7b_400bt_preview'

model_1 = AutoModelForCausalLM.from_pretrained(
    model_path,
    # load_in_8bit=True,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
model_1 = prepare_model_for_kbit_training(model_1)

In [6]:
tokenizer = LlamaTokenizer.from_pretrained(model_path)

# PEFT 설정

In [7]:
from peft import LoraConfig, get_peft_model, TaskType

# PEFT 설정
config = LoraConfig(
    r=4,    # Lora attention dimension / LoRA를 적용하는 가중치 행렬의 수(차원) / 논문에서는 2 4 8의 가중치를 사용하였는데 4인경우에도 충분히 좋은 성능을 보이는 것으로 보여 4 적용
    lora_alpha=16, # 로라 스케일링 값 / README 값 = 32 / r 값을 낮춰적용하면서 같이 낮춰적용함
    target_modules=["q_proj", "v_proj"], # LoRA를 적용할 모듈의 이름
    lora_dropout=2e-5, # dropout 값
    bias="none", # none 혹은 all 혹은 lora_only
    task_type="CAUSAL_LM" # TaskType.SEQ_2_SEQ_LM # task_type="CAUSAL_LM", # task 유형 / 인과관계LM
)

In [8]:
model = get_peft_model(model_1, config)
model.print_trainable_parameters()

trainable params: 2097152 || all params: 6740512768 || trainable%: 0.03111264783824826


# data

In [9]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["instruction"]:
        return f"""
        ### Instruction:
        {data_point["instruction"]}

        ### Input:
        {data_point["input"]}

        ### Response:
        {data_point["output"]}"""

In [10]:
data = load_dataset("json", data_files="/workspace/arrow/no_arrow_test.json")
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-8fb243cff6084934/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 200
    })
})

In [12]:
# pad 값 지정
tokenizer.pad_token_id = 0

# map(함수, 리스트) -> map : 리스트로부터 원소를 하나씩 꺼내 함수를 적용시킨 다음, 그 결과를 새로운 리스트에 담아준다. 
# 데이터 섞은 다음 lambda식 사용해서 tokenizer 사용 후 다시 리스트에 담아주는 것
data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=256,
        padding="max_length",
    )
)
data

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
})

# train

In [13]:
MICRO_BATCH_SIZE = 8
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 2e-5

## 파라미터 변경(업데이트 스텝 수 1)

In [14]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,   # 학습을 위한 GPU/TPU 코어/CPU당 배치 크기. 기본값 8
        gradient_accumulation_steps=1, # 업데이트 스텝 수(GRADIENT_ACCUMULATION_STEPS). 기본값은 1. TrainingArguments 예시에 적용되어있는 값 사용
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,  #  Apex AMP 최적화 수준?
        logging_steps=1, # 두 로그 사이의 업데이트 단계 수? 0 혹은 1이면 단계의 비율로 해석된다는 듯. 
        output_dir="lora-llama",
        save_total_limit=3, # 체크포인트의 총량을 제한한다. 수를 넘으면 이전의 체크포인트를 삭제한다.
    ),

    # data_collator : 데이터 집합 요소 목록을 입력으로 사용하여 배치를 형성하는 개체
    # DataCollatorForLanguageModeling : 언어 모델링에 사용되는 데이터 수집기. 입력이 모두 동일한 크기가 아닌경우 배치의 최대 길이까지 알아서 padding된다.
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [15]:
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,2.837400
2,2.894600
3,2.983400
4,2.893700
5,2.934500
6,2.860800
7,2.820000
8,3.013700
9,3.030500
10,2.797800


RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 19959678144 vs 19959678040

## test

In [16]:
generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)

In [17]:
word_1 = "브라우저 실행"
word_2 = "화면 내려줘"

In [18]:
def prompt(word) :
    return f"""
        ### Instruction:
        Below is an instruction that describes a task, paired with an input that provides context . Write a response that appropriately completes the request

        ### Input:
        {word}

        ### Response:
        """

In [19]:
inputs = tokenizer(prompt(word_1), return_tensors="pt")
input_ids = inputs["input_ids"].cuda()

generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=50,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

<unk>
 ### Instruction:
 Below is an instruction that describes a task, paired with an input that provides context . Write a response that appropriately completes the request

 ### Input:
 브라우저 실행

 ### Response:
 openBrowser()

 ### Context:
 2345 is an invalid URL

 ### Explanation:
 The appropriate response is to prompt for a different URL that can be opened.

### Exercise
Open the Play


In [20]:
inputs = tokenizer(prompt(word_2), return_tensors="pt")
input_ids = inputs["input_ids"].cuda()

generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=50,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

<unk>
 ### Instruction:
 Below is an instruction that describes a task, paired with an input that provides context . Write a response that appropriately completes the request

 ### Input:
 화면 내려줘

 ### Response:
 scrollDown

 ### Context:
 2 inputs are provided below. Use the context to determine what type of input is appropriate for each response

 ### Input #1:
 textInput

 ### Input #2:
 buttonInput



gradient_accumulation_steps ( int, optional , 기본값은 1) — 역방향/업데이트 패스를 수행하기 전에 기울기를 누적할 업데이트 단계 수<br>
업데이트 스텝 수가 너무 작아 1250번의 step으로 훈련하게 됨으로 Disk quota exceeded(디스크 할당량 초과)가 일어남 <br>
하지만 그때 그때 기울기를 조정하며 학습하는 만큼 학습 성과는 잘 나오는 것으로 보인다.

## 파라미터 변경(업데이트 스텝 수 2)

In [24]:
model_2 = get_peft_model(model_1, config)
model_2.print_trainable_parameters()

trainable params: 2097152 || all params: 6740512768 || trainable%: 0.03111264783824826


In [25]:
MICRO_BATCH_SIZE = 8
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 2e-5

In [26]:
trainer = transformers.Trainer(
    model=model_2,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,   # 학습을 위한 GPU/TPU 코어/CPU당 배치 크기. 기본값 8
        gradient_accumulation_steps=2, # 업데이트 스텝 수(GRADIENT_ACCUMULATION_STEPS). 기본값은 1. TrainingArguments 예시에 적용되어있는 값 사용
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,  #  Apex AMP 최적화 수준?
        logging_steps=1, # 두 로그 사이의 업데이트 단계 수? 0 혹은 1이면 단계의 비율로 해석된다는 듯. 
        output_dir="lora-llama",
        save_total_limit=3, # 체크포인트의 총량을 제한한다. 수를 넘으면 이전의 체크포인트를 삭제한다.
    ),

    # data_collator : 데이터 집합 요소 목록을 입력으로 사용하여 배치를 형성하는 개체
    # DataCollatorForLanguageModeling : 언어 모델링에 사용되는 데이터 수집기. 입력이 모두 동일한 크기가 아닌경우 배치의 최대 길이까지 알아서 padding된다.
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model_2.config.use_cache = False
trainer.train()

Step,Training Loss
1,2.866700
2,2.943100
3,2.907300
4,2.932300
5,2.934800
6,2.838500
7,2.939900
8,2.896900
9,2.801800
10,2.862700


RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 19959678144 vs 19959678040

## test

In [28]:
generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)

In [29]:
word_1 = "브라우저 실행"
word_2 = "화면 내려줘"

In [30]:
def prompt(word) :
    return f"""
        ### Instruction:
        Below is an instruction that describes a task, paired with an input that provides context . Write a response that appropriately completes the request

        ### Input:
        {word}

        ### Response:
        """

In [31]:
inputs = tokenizer(prompt(word_1), return_tensors="pt")
input_ids = inputs["input_ids"].cuda()

generation_output = model_2.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=50,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

<unk>
 ### Instruction:
 Below is an instruction that describes a task, paired with an input that provides context . Write a response that appropriately completes the request

 ### Input:
 브라우저 실행

 ### Response:
 openBrowser()

### Exercise 2:

XWalkViewController provides a simple web view for viewing web pages. The controller has a property called url which represents the URL of the page to be displayed. Write a method that returns


In [32]:
inputs = tokenizer(prompt(word_2), return_tensors="pt")
input_ids = inputs["input_ids"].cuda()

generation_output = model_2.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=50,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

<unk>
 ### Instruction:
 Below is an instruction that describes a task, paired with an input that provides context . Write a response that appropriately completes the request

 ### Input:
 화면 내려줘

 ### Response:
 scrollDown()

 ### Context:
 달성하기

### Exercises

Write an expression that returns true if the input is greater than or equal to 10.



gradient_accumulation_steps ( int, optional , 기본값은 1) — 역방향/업데이트 패스를 수행하기 전에 기울기를 누적할 업데이트 단계 수<br>
업데이트 스텝 수가 너무 작아 600번의 step으로 훈련하게 됨으로 Disk quota exceeded(디스크 할당량 초과)가 일어남 <br>
하지만 그때 그때 기울기를 조정하며 학습하는 만큼 학습 성과는 잘 나오는 것으로 보인다.<br>
runpod 내에서 디스크 한계는 499

## 파라미터 변경(업데이트 스텝 수 4)

In [36]:
model_4 = get_peft_model(model_1, config)
model_4.print_trainable_parameters()

trainable params: 2097152 || all params: 6740512768 || trainable%: 0.03111264783824826


In [37]:
MICRO_BATCH_SIZE = 8
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 2e-5

In [38]:
trainer = transformers.Trainer(
    model=model_4,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,   # 학습을 위한 GPU/TPU 코어/CPU당 배치 크기. 기본값 8
        gradient_accumulation_steps=4, # 업데이트 스텝 수(GRADIENT_ACCUMULATION_STEPS). 기본값은 1. TrainingArguments 예시에 적용되어있는 값 사용
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,  #  Apex AMP 최적화 수준?
        logging_steps=1, # 두 로그 사이의 업데이트 단계 수? 0 혹은 1이면 단계의 비율로 해석된다는 듯. 
        output_dir="lora-llama",
        save_total_limit=3, # 체크포인트의 총량을 제한한다. 수를 넘으면 이전의 체크포인트를 삭제한다.
    ),

    # data_collator : 데이터 집합 요소 목록을 입력으로 사용하여 배치를 형성하는 개체
    # DataCollatorForLanguageModeling : 언어 모델링에 사용되는 데이터 수집기. 입력이 모두 동일한 크기가 아닌경우 배치의 최대 길이까지 알아서 padding된다.
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model_4.config.use_cache = False
trainer.train()

Step,Training Loss
1,2.905500
2,2.923600
3,2.895000
4,2.931800
5,2.851100
6,3.025600
7,2.989700
8,2.877500
9,2.894900
10,2.936400


TrainOutput(global_step=300, training_loss=1.2675304627418518, metrics={'train_runtime': 626.3546, 'train_samples_per_second': 15.965, 'train_steps_per_second': 0.479, 'total_flos': 9.74601697886208e+16, 'train_loss': 1.2675304627418518, 'epoch': 48.0})

## test

In [39]:
generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)

In [40]:
word_1 = "브라우저 실행"
word_2 = "화면 내려줘"

In [41]:
def prompt(word) :
    return f"""
        ### Instruction:
        Below is an instruction that describes a task, paired with an input that provides context . Write a response that appropriately completes the request

        ### Input:
        {word}

        ### Response:
        """

In [42]:
inputs = tokenizer(prompt(word_1), return_tensors="pt")
input_ids = inputs["input_ids"].cuda()

generation_output = model_4.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=50,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

<unk>
 ### Instruction:
 Below is an instruction that describes a task, paired with an input that provides context . Write a response that appropriately completes the request

 ### Input:
 브라우저 실행

 ### Response:
 typing_response = input('> ')

 ### Desired Output:
 typing_response

### Exercise Solution
1. input('> ')
2. typing_response = input('> ')



In [43]:
inputs = tokenizer(prompt(word_2), return_tensors="pt")
input_ids = inputs["input_ids"].cuda()

generation_output = model_4.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=50,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

<unk>
 ### Instruction:
 Below is an instruction that describes a task, paired with an input that provides context . Write a response that appropriately completes the request

 ### Input:
 화면 내려줘

 ### Response:
 GetScreenUp

### Step 2: Parse

Next you will need to determine what API calls are appropriate based on the information provided in the request.

* Determine the appropriate API call based on the input and desired


# 궁금함. 업데이트 수를 작게하고 체크포인트 총량을 낮추면 디스크 아웃이 일어나지 않을까?
이해하기로는 설정된 턴 수를 넘으면 이전의 체크포인트를 삭제한다고 하는 것 같으니<br>
쌓이는 대로 바로바로 체크포인트를 삭제해버리면? 499 step 을 넘길 수 있을까?

In [33]:
model_test = get_peft_model(model_1, config)
model_test.print_trainable_parameters()

trainable params: 2097152 || all params: 6740512768 || trainable%: 0.03111264783824826


In [34]:
MICRO_BATCH_SIZE = 8
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 2e-5

In [35]:
trainer = transformers.Trainer(
    model=model_test,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,   # 학습을 위한 GPU/TPU 코어/CPU당 배치 크기. 기본값 8
        gradient_accumulation_steps=2, # 업데이트 스텝 수(GRADIENT_ACCUMULATION_STEPS). 기본값은 1. TrainingArguments 예시에 적용되어있는 값 사용
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,  #  Apex AMP 최적화 수준?
        logging_steps=1, # 두 로그 사이의 업데이트 단계 수? 0 혹은 1이면 단계의 비율로 해석된다는 듯. 
        output_dir="lora-llama",
        save_total_limit=1, # 체크포인트의 총량을 제한한다. 수를 넘으면 이전의 체크포인트를 삭제한다.
    ),

    # data_collator : 데이터 집합 요소 목록을 입력으로 사용하여 배치를 형성하는 개체
    # DataCollatorForLanguageModeling : 언어 모델링에 사용되는 데이터 수집기. 입력이 모두 동일한 크기가 아닌경우 배치의 최대 길이까지 알아서 padding된다.
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model_2.config.use_cache = False
trainer.train()

Step,Training Loss
1,2.866700
2,2.943100
3,2.906900
4,2.931400
5,2.934500
6,2.837600
7,2.938700
8,2.895800
9,2.801000
10,2.861500


RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 19959678144 vs 19959678040

아니였다. 상관이 없었다.